In [1]:
import numpy as np
import pandas as pd

In [2]:
# create a list
pbps = []
# for each season between 2003 and 2010
for season in range(2003,2010):
    # read the play by play file for that corresponding season into a dataframe, add a 'season' column, and add that dataframe to our list
    pbps.append(pd.read_parquet(f'cfbfastr_pbp/play_by_play_{season}.parquet').assign(season=season))
# combine all those dataframes into one
pbp = pd.concat(pbps)

In [3]:
# want to restrict to runs and passes but first let's see what play types will be eliminated by filtering in the way I plan to
eliminated_plays = pbp.loc[(pbp['start_posteam_name']=='USC')&(pbp['pass'].isna())&(pbp['rush'].isna())]
eliminated_plays['play_type_original'].value_counts()

Penalty                       445
Extra Point Good              400
Punt                          255
Timeout                       248
Kickoff                       237
Field Goal Good                94
Punt Return                    86
Pass Interception              67
Kickoff Return (Offense)       62
Fumble Recovery (Opponent)     23
Field Goal Missed              14
Fumble Recovery (Own)          14
Unknown                        13
Extra Point Missed             13
Blocked Punt                    3
Safety                          3
Fumble Return Touchdown         1
Blocked PAT                     1
Two Point Pass                  1
Kickoff Return Touchdown        1
Kickoff Return (Defense)        1
Blocked Field Goal              1
Punt Return Touchdown           1
2pt Conversion                  1
Name: play_type_original, dtype: int64

In [4]:
# as you can see it's not perfect but we're eliminating ~24.5% of plays overall and most of them are ST 
eliminated_plays['play_type_original'].shape[0]/pbp.loc[(pbp['start_posteam_name']=='USC')].shape[0]

0.24491054904380014

In [10]:
# restrict to runs and passes, where USC is on offense, early downs only, WP between 0.2 and 0.8, >=120s left in half at start of play
# adding .copy() at the end for reasons that are complicated but have to do with the fact we're going to create a new column 
usc = pbp.loc[(pbp['pass'].notna() | pbp['rush'].notna()) & (pbp['start_posteam_name']=='USC') & ((pbp['down_1']==1) | (pbp['down_2']==1)) & (pbp['wp_before'] >= 0.2) & (pbp['wp_before'] <= 0.8) & (pbp['start_half_seconds_remaining'] >= 120)].copy()
# note: this dataset has separate 'down_1', 'down_2',... variables which are each 0 (no) or 1 (yes) instead of a single 'down' variable. not sure why

In [11]:
# see how many plays we have per season as a sanity check
usc.groupby('season').agg({'id':'count'})

,id
season,
2003,194
2004,250
2005,287
2006,283
2007,287
2008,184
2009,299


In [15]:
# create a new variable which is 1 for passes, 0 for non passes (ie rushes)
usc['pass_rate'] = np.where(usc['pass'],1,0)
# create one for rushes too so we can make sure things add up to 1.0
usc['rush_rate'] = np.where(usc['rush'],1,0)
# aggregate and sum
usc.groupby('season').agg({'pass_rate':'mean','rush_rate':'mean'}).assign(sum=lambda x:x['pass_rate']+x['rush_rate'])

,pass_rate,rush_rate,sum
season,,,
2003,0.556701,0.443299,1.0
2004,0.476000,0.524000,1.0
2005,0.494774,0.505226,1.0
2006,0.508834,0.491166,1.0
2007,0.480836,0.519164,1.0
2008,0.510870,0.489130,1.0
2009,0.441472,0.558528,1.0
